Insert/overwrite geometry information like this:
```bash
cp sampling_output.h5 sampling_output.nxs
python mcstas_to_nexus_geometry.py config3.instr.xml sampling_output.nxs -v
```

In [ ]:
import scippneutron as scn
import scippnexus as snx
import scipp as sc

In [ ]:
sampled_nexus_file_path = ""
# sampled_nexus_file_path = "../sampling_output.nxs"
xml_file_path = ""  # Either .xml file or h5 file from mcstas simulation.
# xml_file_path = "../original_files/config3.instr.xml"

In [ ]:
dg = snx.load(sampled_nexus_file_path, root="entry/instrument")

In [ ]:
from matplotlib import pyplot as plt

# Display all plots in one row
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
norm_mode = "log"
fig.suptitle(f"Detector Panels 2D ({norm_mode} scale)")
for i_ax, ax in enumerate(axes):
    dg["detector_panel_" + str(i_ax)]["data"].hist().plot(
        norm=norm_mode,
        coords=["y_pixel_offset", "x_pixel_offset"],
        aspect="equal",
        ax=ax,
        title=f"Detector Panel {i_ax}",
    )

plt.tight_layout()
plt.show()

In [ ]:
det_pos_grs = [
    snx.compute_positions(dg[f"detector_panel_{detector_i}"]) for detector_i in range(3)
]
det_pos_das = [det_pos_gr["data"] for det_pos_gr in det_pos_grs]
flattened_das = [
    det_pos_da.flatten(
        dims=["y_pixel_offset", "x_pixel_offset"],
        to="detector_number",
    )
    for det_pos_da in det_pos_das
]
flattened_da = sc.concat(flattened_das, dim="detector_number")

In [ ]:
scn.instrument_view(flattened_da.hist()[::30], norm="log")

## Validation of the Geometry

Checking if the inserted geometry agrees with the parsed pixel positions from xml.

In [ ]:
nexus_da = flattened_da.hist()
nexus_da.values = nexus_da.coords["detector_number"].values
scn.instrument_view(nexus_da[::30])

In [ ]:
import sys

sys.path.append("../")

from mcstas_geometry_xml import read_mcstas_geometry_xml

geo = read_mcstas_geometry_xml(xml_file_path)
meta = geo.detector_metadata(*[det.name for det in geo.detectors])
mcstas_detector_numbers_with_pos = geo.pixel_ids(
    *[det.name for det in geo.detectors]
).rename_dims(id="detector_number")
mcstas_positions = meta["position"].flatten(dims=["panel", "id"], to="detector_number")
mcstas_detector_numbers_with_pos = sc.DataArray(
    data=sc.array(
        dims=["detector_number"], values=mcstas_detector_numbers_with_pos.values
    ),
    coords={
        "position": mcstas_positions,
        "detector_number": mcstas_detector_numbers_with_pos,
    },
)
display(mcstas_detector_numbers_with_pos)

scn.instrument_view(mcstas_detector_numbers_with_pos[::30])

In [ ]:
from scipp.testing import assert_allclose

# Will raise an error if not close enough
assert_allclose(
    mcstas_positions, flattened_da.coords["position"], atol=sc.scalar(1e-14, unit="m")
)